# Working with numerical data - Part I

This notebook accompanies the script <strong><span style="color:red;">03_Numpy_partA.pdf</span></strong>  and provides practical examples related to its content.

In [ ]:
import numpy as np

In [ ]:
print(f"NumPy version {np.__version__}") # Dunder Attribut / Magic Attribut
print(f"NumPy location {np.__file__}")

<hr style="border: none; height: 20px; background-color: green;">

# 2. Why Numerical Data?

Real-world data must be transformed into numerical form to enable efficient computation.

## Examples:
- Audio → Amplitude values over time
- Images → Pixel intensity matrices
- Biological signals → Time series
- Genomics → One-hot encoded sequences

NumPy provides a unified structure for such data.

<hr style="border: none; height: 10px; background-color: CornflowerBlue;">

### Example: Audio - Gooose

In [ ]:
from scipy.io import wavfile
from IPython.display import Audio
import matplotlib.pyplot as plt

rate, data = wavfile.read("../data/audio/gooose.wav")

print(rate)        # Sampling-Rate (Hz)
print(data.shape)  # Samples
print(type(data))  # Datentyp
print(data.dtype)  # Datentyp

Audio(data, rate=rate)

#### Show complete File

In [ ]:
rate, data = wavfile.read("../data/audio/gooose.wav")

t = np.arange(len(data)) / rate

plt.plot(t, data)
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.title("Audio Waveform")
plt.show()

#### Show short time period

In [ ]:
# Time window
start_s = 5.0
duration_s = 0.01

# Convert to samples
start = int(start_s * rate)
end = start + int(rate * duration_s)

# Slice
data_part = data[start:end]

# Time axis
t = np.arange(len(data_part)) / rate + start_s

# Plot
plt.plot(t, data_part, marker=".")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.title(f"Audio Waveform ({start_s} - {start_s+duration_s} s)")
plt.grid(True, axis="y")
plt.show()

#### Compute the frequency spectrum and the corresponding frequencies in Hz

In [ ]:
# Load audio
rate, data = wavfile.read("../data/audio/gooose.wav")

# Convert to float
arr = data.astype(float)

# Remove DC offset (mean)
centered_signal = arr - arr.mean()

# Optional: windowing (reduces spectral leakage)
window = np.hanning(len(centered_signal))
processed_signal = centered_signal * window

# Fast Fourier Transform
fft = np.fft.rfft(processed_signal)
freq = np.fft.rfftfreq(len(processed_signal), d=1/rate)

# Magnitude
magnitude = np.abs(fft)

# Plot
plt.figure()
plt.plot(freq, magnitude)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude")
plt.title("Frequency Spectrum")
plt.show()


<hr style="border: none; height: 10px; background-color: CornflowerBlue;">

### Example: ECG – Real Biomedical Signals

In [ ]:
import wfdb

# Load the record (reads .hea and .dat)
record = wfdb.rdrecord('../data/ecg_db/patient001/s0010_re')

# Access all channels as a NumPy array
record.p_signal[0:3]

In [ ]:
# Transpose: (channels, samples)
channels = record.p_signal.T

selected_leads = [
    "i", "ii", "iii", "avr", "avl", "avf",
    "v1", "v2", "v3", "v4", "v5", "v6"
]

# Find indices of selected leads
lead_indices = [
    i for i, name in enumerate(record.sig_name)
    if name.lower() in selected_leads
]

# Extract selected channels
channels_12 = channels[lead_indices]

# Corresponding lead names
lead_names = [record.sig_name[i] for i in lead_indices]

# Define time window (in samples)
start_idx = 1300
end_idx   = 2600

fs = record.fs  # sampling rate (Hz)

# Time axis in seconds
t = np.arange(start_idx, end_idx) / fs

# Define lead groups
inferior_leads = ["ii", "iii", "avf"]

# Plot inferior leads
plt.figure(figsize=(15, 10))
fontsize = 20

for name, signal in zip(lead_names, channels_12):
    # Plot only inferior leads
    if name.lower() in inferior_leads:

        plt.plot(
            t,
            signal[start_idx:end_idx],
            label=name.upper()
        )

plt.xlabel("Time (s)", fontsize=fontsize)
plt.ylabel("Amplitude (mV)", fontsize=fontsize)
plt.xticks(fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.title(
    "ECG Signal - Inferior Leads – Patient 001",
    fontsize=28
)
plt.legend(
    loc="upper right",
    fontsize=fontsize - 4,
    title="Leads",
    title_fontsize=fontsize - 2
)
plt.grid(True, alpha=0.3)
plt.tight_layout()

<hr style="border: none; height: 10px; background-color: CornflowerBlue;">

### Example: Image - Cat

In [ ]:
from PIL import Image

img = Image.open("../data/images/hugo.jpg")
arr = np.array(img)

arr.shape

In [ ]:
plt.imshow(arr)
plt.axis("off");

<hr style="border: none; height: 20px; background-color: green;">

# 3. Creating NumPy Arrays

### From Python Lists

Creating a NumPy array from a python list `[1, 2, 3, 4, 5]`

In [ ]:
arr = np.array([1, 2, 3, 4, 5])
arr

### Explicit Data Type

We can optionally set the data type explicitly

In [ ]:
arr_float = np.array([1, 2, 3, 4, 5], dtype=np.float32)
arr_float


## 4. Array Attributes

Every NumPy array exposes metadata that describes its structure.



| Attribute  | Meaning                          |
|------------|----------------------------------|
| dtype      | Data type of elements            |
| ndim       | Number of dimensions             |
| shape      | Size per dimension               |
| strides    | Memory step size                 |
| size       | Total number of elements         |
| itemsize   | Bytes per element                |


In [ ]:
arr = np.array([[0,1,2],[3,4,5],[6,7,8]], dtype=np.float32)

print("dtype:", arr.dtype)
print("ndim:", arr.ndim)
print("shape:", arr.shape)
print("strides:", arr.strides)
print("size:", arr.size)
print("itemsize:", arr.itemsize)


## 5. Creating Arrays from Scratch

NumPy provides fast constructors for common initialization patterns.


### Zeros and Ones

We can initialize and create new arrays, that are for example filled with zeros or ones

In [ ]:
np.zeros(10, dtype=int)

In [ ]:
np.ones((3,5), dtype=float)

### Constant Values

Also, we can create new arrays that are filled with a given value:

In [ ]:
np.full((3,5), 3.14)

### Ranges

With `np.arange()`, we can generate evenly spaced values within a given range, similar to Python’s built-in `range()`, but it returns a NumPy array

In [ ]:
# Syntax
np.arange(start=0, stop=10, step=1, dtype=None)

In [ ]:
arr = np.arange(10)
print("Array from 0 to 9:", arr)

In [ ]:
# Creating an array from 3 to 10
arr = np.arange(3, 10)
print("Array from 3 to 10:", arr)

# Creating an array from 5 to 15 with a step of 2
arr_step = np.arange(5, 15, 2)
print("Array from 5 to 15 with step 2:", arr_step)

# Creating an array with floating-point numbers
arr_float = np.arange(0, 1, 0.2, dtype=np.float32)
print("Array with float step 0.2:", arr_float)

# Creating a descending array
arr_desc = np.arange(10, 0, -2)
print("Descending array:", arr_desc)

# Combining np.arange() with reshape()
arr = np.arange(16).reshape(4, 4)
print("2dim array:", arr)

# Creating an array from 0 to 9
arr = np.arange(10)
print("Array from 0 to 9:", arr)


## 6. Data Types (dtype)

Choosing the correct dtype is important for:
- Memory efficiency
- Numerical precision
- Performance


In [ ]:
# When constructing an array, we can specify its data type using a string:
np.array([1.5, 2.9], dtype=int)

In [ ]:
np.zeros(10, dtype='int16')

In [ ]:
# Or the associated numpy data type (recommended):
np.zeros(10, dtype=np.int16)


**Best practice:** Prefer `np.int16`, `np.float32`, etc. over string-based types.



## 7. Type Casting and Upcasting

NumPy handles mixed types differently depending on the operation.

While Python lists can contain multiple data types.  
NumPy arrays contain only one type.

If types do not match, NumPy will upcast if possible.

### Integers are up-cast to floating point:

In [ ]:
# Integers are up-cast to floating point:
arr = np.array([3.14, 4, 2, 3])
print(arr)

### Arrays can only contain one data type

We cannot mix different numerical data types in the same array.

In [ ]:
# Create a 2D NumPy array
arr = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])
print(arr)

If we try to add a float, it will be automatically truncated to an int.

In [ ]:
# Modify one element
arr[1, 0] = 9.9
print(arr)

### New Array (Upcasting)

In [ ]:
arr = np.array([1,2])
arr + 1.9  # Creates an new array

### Adding float to int array (in-place)

In [ ]:
arr = np.array([1, 2])

try:
    arr += 1.9 # changes values in-place
except TypeError as e:
    print("In-place addition failed due to dtype casting. Converting array to float.")
    print("Original error:", e)

    arr = arr.astype(float)
    arr += 1.9

arr


## 8. Dimensions and Shapes

Arrays can have arbitrary dimensions (tensors).


In [ ]:
arr = np.random.rand(2,3,4)
arr.shape


Example: PyTorch images usually have shape:

```
(batch, channel, height, width)
```



## 9. Indexing

### 1D Indexing


In [ ]:
arr = np.array([5,0,3,3,7,9])

In [ ]:
arr[0] # Indexing from the beginning of the array

In [ ]:
arr[-1] # Indexing from the end of the array

### 2D Indexing

In [ ]:
arr = np.array([[3,5,2,4],
               [7,6,8,8],
               [1,6,7,7]])

print(arr)

#### Examples

In [ ]:
# What will this give?
print(arr[0,0])

In [ ]:
# What will this give?
arr[2,-1]

## 10. Iteration

### Define a NumPy Array

In [ ]:
arr = np.array([
    [0.3, 0.4, 0.8],
    [0.5, 0.6, 0.7]
])
print(arr)

### Explicit Row-Column Iteration in a NumPy Array

In [ ]:
rows, cols = arr.shape

for i in range(rows):
    for j in range(cols):
        print(i, j, arr[i, j])

### Using shape to get Rows and Columns

In [ ]:
# Get rows and columns using shape
print(f"shape: {arr.shape}")  # (2, 3)

rows, columns = arr.shape
print(f"Rows: {rows}, Columns: {columns}")

### Iteration: Cleaner and more Pythonic with enumerate()

A cleaner and more Pythonic approach to iteration is to use `enumerate()`,
which eliminates the need for manual indexing, improving both readability and efficiency


In [ ]:
for i, row in enumerate(arr):          # i = row index, row = full row
    for j, value in enumerate(row):    # j = column index, value = element
        print(i, j, value)

### Iteration using np.ndenumerate

NumPy provides its own optimized iteration method: `np.ndenumerate()`.  
This approach is more compact, works seamlessly with arrays of any dimension and is particularly efficient for NumPy arrays.   
By using NumPy’s built-in optimizations, it allows for faster and more readable iteration compared to traditional looping methods.

In [ ]:
for (i, j), value in np.ndenumerate(arr):
    print(i, j, value)


## 11. Slicing

Slicing allows you to extract specific parts of a NumPy array using the `[start:stop:step]` syntax  

```
[start:stop:step]
```


In [ ]:
arr = np.arange(10)
print(arr)

In [ ]:
arr[:5] # First five elements

In [ ]:
arr[5:] # Elements after index 5

In [ ]:
arr[4:7] # Middle sub-array (index 4 to 6)

In [ ]:
arr[::2] # Every other element

In [ ]:
arr[1::2] # Every other element, starting at index 1

In [ ]:
arr_2d = np.array([[1, 2, 3, 4],
               [5, 6, 7, 8]])

arr_2d[:,1:3] # Select all rows and columns 1 to 2 (slice 1:3, end exclusive)

### Array Slicing: Extracting Subarrays from Multi-Dimensional Arrays

Create a 5×5 NumPy array with increasing integers:

```
array([[ 0,  1,  2,  3,  4,  5],
       [10, 11, 12, 13, 14, 15],
       [20, 21, 22, 23, 24, 25],
       [30, 31, 32, 33, 34, 35],
       [40, 41, 42, 43, 44, 45],
       [50, 51, 52, 53, 54, 55]])
```

In [ ]:
# Option 1
arr = np.arange(6) + np.arange(0, 60, 10).reshape(6, 1)

# Option 2
arr = np.array([[row * 10 + col for col in range(6)] for row in range(6)])

print(arr)

In [ ]:
arr[0,3:5]

In [ ]:
arr[4:,4:]

In [ ]:
arr[:,2]

In [ ]:
arr[2::2,::2]

### What Happens When We Assign a Slice to a New Variable?

In [ ]:
arr_sub = arr[0,3:5]
arr_sub

In [ ]:
# Create a 3x4 NumPy array
arr = np.array([
    [12, 5, 2, 4],
    [7, 6, 8, 8],
    [1, 6, 7, 7]
])

# Print the original array
print("Original array (arr):")
print(arr)

In [ ]:
# Extract a 2x2 subarray
arr_sub = arr[:2, :2]

print("Extracted 2x2 subarray:")
print(arr_sub)

In [ ]:
# Modify the subarray
arr_sub[0, 0] = 99
print(arr_sub)

In [ ]:
# Print the original array again (shows that slicing creates a view)
print(arr)


## 12. Views vs Copies

Slicing usually returns a *view*, not a copy.


### View Example

In [ ]:
arr = np.arange(12).reshape(4,3)
print(arr)

In [ ]:
view = arr[:2,:2]
view[:] = 77
print(view)

In [ ]:
print(arr)

### Copy Example

In [ ]:
arr = np.arange(12).reshape(4,3)


In [ ]:
copy = arr[:2,:2].copy()
copy[:] = 77
print(copy)

In [ ]:
print(arr)


Use `.copy()` when you need independent data.



## 13. Memory Ownership (.base)

The `.base` attribute shows the original owner of the data.


In [ ]:
arr = np.arange(0, 20, 2)
print(arr)

In [ ]:
arr_view = arr.view() # .view() creates a view of our numpy array
arr_view.base is arr # .base will tell us which is the base of the array

In [ ]:
arr_view.base is arr_view # A view is not the base of itself

In [ ]:
arr_view.flags.owndata # True if the array owns its data, False if it is just a view on another array

#### Copy of an array: base

In [ ]:
arr_copy = arr.copy()
arr_copy.base is arr

In [ ]:
arr_copy.base is None

In [ ]:
arr_copy.flags.owndata

### Understanding `deepcopy()` in NumPy

If your NumPy array contains nested objects (e.g., lists, dictionaries), use `copy.deepcopy()` instead of `.copy()` to avoid unexpected modifications

In [ ]:
import copy

# A NumPy array with lists as elements (dtype=object)
arr_1 = np.array([[1, [2, 3]], [4, [5, 6]]], dtype=object)

# Normal .copy() — NOT a true deep copy!
arr_2 = arr_1.copy()

# copy.deepcopy() — A TRUE deep copy!
arr_3 = copy.deepcopy(arr_1)

# Modifying an element inside a nested list
arr_2[0, 1][0] = 99  # This change also affects the original array!

print("Original array (arr_1):")
print(arr_1)  # The original is modified due to changes in 'b'!

print("\nNormal copy (arr_2):")
print(arr_2)  # Reflects the changes

print("\nDeep copy (arr_3):")
print(arr_3)  # Remains unchanged


## 14. Concatenation

Combine arrays along existing axes.


In [ ]:
arr_1 = np.array([0.23, 0.73, 0.38])
arr_2 = np.array([0.82, 0.12, 0.95])

np.concatenate([arr_1, arr_2])

In [ ]:
arr_3 = np.array([0.41, 0.66])
np.concatenate([arr_1, arr_2, arr_3])

#### Concatenating Two-Dimensional Arrays

In [ ]:
arr_1 = np.array([[1, 2, 3], [4, 5, 6]])
arr_2 = np.array([[7, 8, 9]])

arr = np.concatenate((arr_1, arr_2))
print(arr)

In [ ]:
arr_1 = np.array([[1, 2], [3, 4], [5, 6]])
arr_2 = np.array([[5, 6, 7], [8, 9, 10], [11, 12, 13]])

arr_3 = np.concatenate((arr_1, arr_2), axis=1)
print(arr_3)

## 15. Splitting

Split arrays into multiple parts.

In [ ]:
arr = np.arange(6) + np.arange(0, 60, 10).reshape(6, 1)
print(arr)

#### Vertical splitting


In [ ]:
arr_1, arr_2, arr_3 = np.vsplit(arr, 3)
print(arr_1, arr_2, arr_3, sep="\n\n")

#### Horizontal splitting

In [ ]:
arr_1, arr_2, arr_3 = np.hsplit(arr, 3)
print(arr_1, arr_2, arr_3, sep="\n\n")


## 16. Vectorization and Performance

Vectorized operations are implemented in optimized C code.
They are significantly faster than Python loops.


In [ ]:
import math

arr = np.random.rand(1_000_000)

def sqrt_loop(values):
    output = np.empty(len(arr))
    for i in range(len(arr)):
        output[i] = math.sqrt(arr[i])
    return output

# Measure execution time with loop
%timeit sqrt_loop(arr)


## 17. UFunc Output Parameter

- NumPy provides two main types of ufuncs: unary and binary
- Unary ufuncs operate on a single input array element-wise, such as `np.sqrt()`
- Binary ufuncs take two input arrays and perform element-wise operations, like `np.add()`
- In addition to unary and binary element-wise operations, NumPy offers reductions, linear algebra, and sorting functions that operate on entire arrays


### Unary UFuncs (operate on a single input):

In [ ]:
arr = np.array([1, 7, 6, 4])

print(f"np.abs(arr) :{np.abs(arr)}")   # Absolute value
print(f"np.exp(arr) :{np.exp(arr)}")   # Exponential function
print(f"np.sqrt(arr):{np.sqrt(arr)}")  # Square root

### Binary UFuncs (operate on two inputs):

In [ ]:
arr_1 = np.array([1, 2, 3])
arr_2 = np.array([4, 5, 6])

print(arr_1 + arr_2)    # Element-wise addition
print(arr_1 * arr_2)    # Element-wise multiplication
print(arr_1 ** arr_2)   # Exponentiation

When working with Ufuncs, specifying an output array using the out parameter can improve efficiency by avoiding unnecessary memory allocations  
Note: Python lists do not have an equivalent to the`out` parameter

In [ ]:
arr_1 = np.arange(5)

# Preallocate output array and write results in-place
arr_2 = np.empty_like(arr_1, dtype=np.float64)  # Ensure 'arr_2' is a float array
np.sqrt(arr_1, out=arr_2)                       # Write results directly into 'arr_2'

#### Understanding `np.empty()` and `np.empty_like()`

Unlike Python lists, NumPy allows creating uninitialized arrays, meaning their contents are random memory values

In [ ]:
arr = np.empty(5, dtype=float)
print(arr)  # Memory content, random values


## 18. Aggregations

The plot shows the probability density of daily average temperatures in 2024 for Basel and Jungfraujoch.   
Basel exhibits a much warmer distribution with higher mean temperatures, while Jungfraujoch is centered around sub-zero values, reflecting its high-altitude alpine climate.  

In [ ]:
from scipy.stats import gaussian_kde

# Load tavg column (index 1)
basel = np.genfromtxt(
    "../data/csv/basel_2024.csv", delimiter=",", skip_header=1, usecols=1
)
jung = np.genfromtxt(
    "../data/csv/jungfraujoch_2024.csv", delimiter=",", skip_header=1, usecols=1
)

# Remove NaNs
basel = basel[~np.isnan(basel)]
jung = jung[~np.isnan(jung)]

# KDE
kde_basel = gaussian_kde(basel)
kde_jung = gaussian_kde(jung)

x_values = np.linspace(-30, 35, 500)

# Plot
plt.figure(figsize=(12, 7))
plt.plot(x_values, kde_basel(x_values), label="Basel")
plt.plot(x_values, kde_jung(x_values), label="Jungfraujoch")

# Mean lines
plt.axvline(basel.mean(), linestyle="--",
            label=f"Mean Basel: {basel.mean():.1f}°C")
plt.axvline(jung.mean(), linestyle="--",
            label=f"Mean Jungfraujoch: {jung.mean():.1f}°C")

plt.title("Temperature Distribution in 2024: Basel vs. Jungfraujoch")
plt.xlabel("Temperature (°C)")
plt.ylabel("Density")
plt.legend()
plt.grid(True)

plt.show()


### An Example with summing values of an array

We create a random array of 1000 items

In [ ]:
arr = np.random.random(1_000)
big_arr = np.random.random(1_000_000)

We can compute the sum of all items of the array with built-in python `sum()` function

In [ ]:
%timeit sum(arr)
%timeit sum(big_arr)

Or with numpy `np.sum()`   
NumPy’s `np.sum()` is orders of magnitude faster than Python’s `sum()`, especially for large arrays

In [ ]:
%timeit np.sum(arr)
%timeit np.sum(big_arr)

### Operations on multidimensional arrays

In [ ]:
arr = np.random.random((3, 4))
print(arr)

In [ ]:
arr.sum(axis=1)

In [ ]:
arr.sum(axis=0)

# 19. More Examples from the Table on Slide 74 (How Do We Aggregate a Dataset?)

Creating Example Data to play with aggregating datasets

In [ ]:
rng = np.random.default_rng(42)

# 2D dataset: 100 rows, 5 features
arr = rng.normal(loc=10, scale=3, size=(100, 5))

# Inject some NaN values
mask = rng.random(arr.shape) < 0.05 # 5% missing values
arr[mask] = np.nan

arr[:5]

### Basic Aggregations

Mean, Standard Deviation, Variance

In [ ]:
mean = np.mean(arr)
std = np.std(arr)
var = np.var(arr)

mean, std, var

### NaN-Safe Aggregations

NumPy provides nan* variants that ignore NaN values.

In [ ]:
mean_nan = np.nanmean(arr)
std_nan = np.nanstd(arr)
var_nan = np.nanvar(arr)

mean_nan, std_nan, var_nan

### Minimum, Maximum, Sum, Product

These operations can be computed without sorting.

In [ ]:
min_val = np.nanmin(arr)
max_val = np.nanmax(arr)
sum_val = np.nansum(arr)
prod_val = np.nanprod(arr)

min_val, max_val, sum_val, prod_val

### Percentiles and Median

Percentiles and medians require sorting. They are therefore slower and less parallelizable.

In [ ]:
p25 = np.nanpercentile(arr, 25)
p50 = np.nanpercentile(arr, 50) # median
p75 = np.nanpercentile(arr, 75)

median = np.nanmedian(arr)

p25, p50, p75, median

### Aggregation Along Axes

By default, NumPy aggregates over the entire array. You can aggregate per column or per row using axis.

In [ ]:
# Per Column (Feature-wise)
col_mean = np.nanmean(arr, axis=0)
col_std = np.nanstd(arr, axis=0)

col_mean, col_std

In [ ]:
# Per Row (Sample-wise)
row_mean = np.nanmean(arr, axis=1)
row_mean[:10]

### Using the out Parameter

The out parameter writes the result into an existing array. This avoids memory allocation and improves performance.

In [ ]:
out_array = np.empty(5)

np.nanmean(arr, axis=0, out=out_array)

out_array

In [ ]:
# Compare with:
np.nanmean(arr, axis=0)

### Data Types and Precision

The output type depends on the input type.

In [ ]:
arr_int = np.array([1, 2, 3, 4], dtype=np.int32)
arr_float = np.array([1, 2, 3, 4], dtype=np.float32)

np.mean(arr_int).dtype, np.mean(arr_float).dtype

### Performance Experiment

We compare fast (no sorting) and slow (sorting-based) operations.

In [ ]:
big = rng.normal(size=10_000_000)

%timeit np.mean(big)
%timeit np.median(big)